In [18]:
import nascar_api
from pathlib import Path

In [19]:
parent_dir = Path.cwd().parent
inator_dir = parent_dir.joinpath("collect_inator")
inator_dir.mkdir(exist_ok=True)

In [ ]:
for fi, f in enumerate(nascar_api.Feeds):
   #print(fi, f.name, f.value)
   p = f.value.params
   param_string = (", " + ", ".join([f"{x}: int" for x in p])) if p != [] else ""
   # param_string = (", " + ": int, ".join(p) + ": int") if p != [] else ""
   dict_string = ", ".join([f"\"{x}\": {x}" for x in p])
   
   script_string = f"""from toolbox.nascar_api import Feeds, get_api_data
from sqlalchemy import Engine 
import pandas as pd

def get_{f.name.lower()}(url_header: str{param_string}, sql_engine: Engine, load_to_sql: bool = True) -> tuple[bool, pd.DataFrame]:
   api_result = get_api_data(Feeds.{f.name}, {{{dict_string}}}, url_header=url_header)

   if api_result[0]:
      df = api_result[2]

      # cleaning logic goes here if needed
      
      if load_to_sql:
         exist_df = pd
         df.to_sql("{f.name.lower()}", con=sql_engine, if_exists="append", index=False)

      return (True, df)
   else:
      print(f"API call failed for Feeds.{f.name}. URL response code = {{api_result[1]}}")
      return (False, pd.DataFrame())"""
   
   script_dir = inator_dir.joinpath(f"generated_{f.name.lower()}").with_suffix(".py")
   if not script_dir.exists():
      script_dir.touch()
      script_dir.write_text(script_string)